# Modeling9: 다양한 모델의 베이지안 최적화

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 데이터 불러오기
data_origin = pd.read_csv("https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv")
data_origin

,gender,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,...,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married
0,female,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,...,9.0,7.0,7.0,6.0,5.0,0.14,3.0,7.0,6.0,0
1,female,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,...,8.0,7.0,8.0,5.0,6.0,0.54,3.0,7.0,5.0,0
2,female,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,...,8.0,9.0,8.0,5.0,7.0,0.16,3.0,7.0,5.0,1
3,female,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,...,6.0,8.0,7.0,6.0,8.0,0.61,3.0,7.0,6.0,1
4,female,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,...,6.0,7.0,7.0,6.0,6.0,0.21,3.0,6.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,male,25.0,26.0,1.0,10.0,10.0,30.0,20.0,10.0,15.0,...,5.0,5.0,5.0,NaN,NaN,0.64,10.0,2.0,5.0,0
8358,male,25.0,24.0,1.0,50.0,20.0,10.0,5.0,10.0,5.0,...,6.0,8.0,4.0,4.0,NaN,0.71,10.0,4.0,4.0,0
8359,male,25.0,29.0,1.0,40.0,10.0,30.0,10.0,10.0,NaN,...,7.0,8.0,8.0,8.0,NaN,-0.46,10.0,6.0,5.0,0
8360,male,25.0,22.0,1.0,10.0,25.0,25.0,10.0,10.0,20.0,...,6.0,5.0,4.0,NaN,5.0,0.62,10.0,5.0,5.0,0


In [3]:
# 전처리를 위한 원본 데이터 복사
data = data_origin.copy()

In [4]:
# 메모리를 효율적으로 사용하기 위한 downcast 함수 정의
def downcast(df, verbose=True):     # verbose 옵션 추가: (True)인 경우 몇 퍼센트 압축됐는지 출력
    start_mem = df.memory_usage().sum() / 1024**2   # 초기 메모리 사용량
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2

    if verbose:
        print(f'{(100*(start_mem - end_mem) / start_mem):.1f}% 압축됨')

    return df

In [5]:
downcast(data)

49.6% 압축됨


,gender,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,...,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married
0,female,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,...,9.0,7.0,7.0,6.0,5.0,0.14,3.0,7.0,6.0,0
1,female,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,...,8.0,7.0,8.0,5.0,6.0,0.54,3.0,7.0,5.0,0
2,female,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,...,8.0,9.0,8.0,5.0,7.0,0.16,3.0,7.0,5.0,1
3,female,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,...,6.0,8.0,7.0,6.0,8.0,0.61,3.0,7.0,6.0,1
4,female,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,...,6.0,7.0,7.0,6.0,6.0,0.21,3.0,6.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,male,25.0,26.0,1.0,10.0,10.0,30.0,20.0,10.0,15.0,...,5.0,5.0,5.0,NaN,NaN,0.64,10.0,2.0,5.0,0
8358,male,25.0,24.0,1.0,50.0,20.0,10.0,5.0,10.0,5.0,...,6.0,8.0,4.0,4.0,NaN,0.71,10.0,4.0,4.0,0
8359,male,25.0,29.0,1.0,40.0,10.0,30.0,10.0,10.0,NaN,...,7.0,8.0,8.0,8.0,NaN,-0.46,10.0,6.0,5.0,0
8360,male,25.0,22.0,1.0,10.0,25.0,25.0,10.0,10.0,20.0,...,6.0,5.0,4.0,NaN,5.0,0.62,10.0,5.0,5.0,0


In [6]:
data_OHE = pd.get_dummies(data, columns=['gender'], drop_first=True)

> Train/Test data split

In [7]:
# 단순 랜덤 샘플링
#from sklearn.model_selection import train_test_split

#train_set, test_set = train_test_split(data, test_size = 0.2, random_state=42)


# 계층적 샘플링
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(data_OHE, data_OHE['married']):
    sss_train_set = data_OHE.loc[train_index]
    sss_test_set = data_OHE.loc[test_index]

In [8]:
X_train = sss_train_set.drop("married", axis=1)
y_train = sss_train_set["married"].copy()

X_test = sss_test_set.drop("married", axis=1)
y_test = sss_test_set["married"].copy()

# Preprocessing

In [9]:
# KNN 결측치 대체
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3)

X_train_imputed = imputer.fit_transform(X_train)
X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train.columns)

X_test_imputed = imputer.transform(X_test)
X_test_imputed = pd.DataFrame(X_test_imputed, columns=X_test.columns)

In [10]:
# min-max scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)

X_test_scaled = scaler.transform(X_test_imputed)

In [11]:
X_train_scaled

array([[0.2702703 , 0.16216218, 0.8888889 , ..., 0.4       , 0.2       ,
        1.        ],
       [0.1891892 , 0.4054054 , 0.7777778 , ..., 0.6       , 0.4       ,
        1.        ],
       [0.2702703 , 0.13513511, 0.5555556 , ..., 0.6       , 0.5       ,
        1.        ],
       ...,
       [0.24324328, 0.1081081 , 0.22222224, ..., 0.7       , 0.7       ,
        0.        ],
       [0.1891892 , 0.43243247, 0.        , ..., 0.6       , 0.6       ,
        1.        ],
       [0.24324328, 0.13513511, 0.7777778 , ..., 0.6       , 0.4       ,
        1.        ]], dtype=float32)

### Over-Sampling

In [12]:
# SMOTE-Tomek 샘플링
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks

smoteto = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
X_resampled, y_resampled = smoteto.fit_resample(X_train_scaled, y_train)

In [13]:
X_resampled.shape, y_resampled.shape

((10956, 32), (10956,))

> 기본적인 데이터셋 준비 완료

# Model Training

### LighGBM

In [14]:
# 베이지안 최적화
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

param_bounds = {
    'n_estimators': (100, 500),
    'max_depth': (3, 10),
    'num_leaves': (20, 100),
    'min_child_samples': (10, 30),
    'learning_rate': (0.01, 0.3),
    'scale_pos_weight': (1, 2),
}

def eval_function(n_estimators, max_depth, num_leaves, min_child_samples, learning_rate, scale_pos_weight):
    params = {
        'n_estimators': int(n_estimators),
        'max_depth': int(max_depth),
        'num_leaves': int(num_leaves),
        'min_child_samples': int(min_child_samples),
        'learning_rate': learning_rate,
        'scale_pos_weight': scale_pos_weight,
    }
    lgbm = LGBMClassifier(**params, objective='binary', metric='accuracy', random_state=42, verbose=0)

    lgbm.fit(X_train_scaled, y_train)
    y_pred = lgbm.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    return acc

In [15]:
from bayes_opt import BayesianOptimization

optimizer = BayesianOptimization(f=eval_function, pbounds=param_bounds, random_state=42)
optimizer.maximize(init_points=5, n_iter=10)

|   iter    |  target   | learni... | max_depth | min_ch... | n_esti... | num_le... | scale_... |
-------------------------------------------------------------------------------------------------
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
| 1         | 0.8506    | 0.1186    | 9.655     | 24.64     | 339.5     | 32.48     | 1.156     |
| 2         | 0.8344    | 0.02684   | 9.063     | 22.02     | 383.2     | 21.65     | 1.97      |
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGB

In [16]:
# 최적화된 하이퍼파라미터로 모델 재학습
best_params = {
    'n_estimators': round(optimizer.max['params']['n_estimators']),
    'max_depth': round(optimizer.max['params']['max_depth']),
    'num_leaves': round(optimizer.max['params']['num_leaves']),
    'min_child_samples': round(optimizer.max['params']['min_child_samples']),
    'learning_rate': optimizer.max['params']['learning_rate'],
    'scale_pos_weight': optimizer.max['params']['scale_pos_weight'],
}

best_lgbm = LGBMClassifier(**best_params, objective='binary', metric='accuracy', random_state=42, verbose=0)
best_lgbm.fit(X_train_scaled, y_train)

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

LGBMClassifier(learning_rate=0.1422602954229404, max_depth=8, metric='accuracy',
               min_child_samples=14, n_estimators=306, num_leaves=67,
               objective='binary', random_state=42,
               scale_pos_weight=1.0464504127199978, verbose=0)

In [17]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Train data 성능 평가
y_train_pred = best_lgbm.predict(X_train_scaled)
print("<<Train Set Performance>>")
print("Accuracy: ", accuracy_score(y_train, y_train_pred))
print("F1 Score: ", f1_score(y_train, y_train_pred))
print(confusion_matrix(y_train, y_train_pred))

# Test data 성능 평가
y_test_pred = best_lgbm.predict(X_test_scaled)
print("<<Test Set Performance>>")
print("Accuracy: ", accuracy_score(y_test, y_test_pred))
print("F1 Score: ", f1_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))

<<Train Set Performance>>
Accuracy:  1.0
F1 Score:  1.0
[[5479    0]
 [   0 1210]]
<<Test Set Performance>>
Accuracy:  0.8595337716676629
F1 Score:  0.532803180914513
[[1304   66]
 [ 169  134]]


### XGBoost

In [18]:
# 베이지안 최적화
from xgboost import XGBClassifier

param_bounds = {
    'eta' : (0.01, 0.3),
    'max_leaves': (2, 1024),
    'n_estimators': (100, 1000),
    'gamma': (0, 10),
    'max_depth': (3, 15),
    'min_child_weight': (1, 10),
    'scale_pos_weight': (1, 3),
    }

def eval_function(max_leaves, eta, n_estimators, gamma, max_depth, min_child_weight, scale_pos_weight):
    params = {
        'eta' : eta, 
        'max_leaves': int(max_leaves),
        'n_estimators': int(n_estimators),
        'gamma': gamma,
        'max_depth': int(max_depth),
        'min_child_weight': min_child_weight,
        'scale_pos_weight': scale_pos_weight,
    }

    xgb_model = XGBClassifier(**params, objective='binary:logistic', random_state=42)
    xgb_model.fit(X_train_scaled, y_train)
    y_pred = xgb_model.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    return acc

In [19]:
from bayes_opt import BayesianOptimization

optimizer = BayesianOptimization(f=eval_function, pbounds=param_bounds, random_state=42)
optimizer.maximize(init_points=5, n_iter=50)

|   iter    |  target   |    eta    |   gamma   | max_depth | max_le... | min_ch... | n_esti... | scale_... |
-------------------------------------------------------------------------------------------------------------


| 1         | 0.8332    | 0.1186    | 9.507     | 11.78     | 613.8     | 2.404     | 240.4     | 1.116     |
| 2         | 0.8362    | 0.2612    | 6.011     | 11.5      | 23.04     | 9.729     | 849.2     | 1.425     |
| 3         | 0.8386    | 0.06273   | 1.834     | 6.651     | 538.3     | 4.888     | 362.1     | 2.224     |
| 4         | 0.8404    | 0.05045   | 2.921     | 7.396     | 468.1     | 8.067     | 279.7     | 2.028     |
| 5         | 0.8476    | 0.1818    | 0.4645    | 10.29     | 176.3     | 1.585     | 954.0     | 2.931     |
| 6         | 0.8482    | 0.2887    | 0.3741    | 13.26     | 174.8     | 9.256     | 952.1     | 2.138     |
| 7         | 0.838     | 0.16      | 0.2326    | 14.37     | 180.7     | 9.329     | 955.0     | 2.551     |
| 8         | 0.8416    | 0.1335    | 3.408     | 13.86     | 484.9     | 9.484     | 367.5     | 2.361     |
| 9         | 0.8422    | 0.2996    | 0.5117    | 12.39     | 760.7     | 8.131     | 414.4     | 1.929     |
| 10      

In [20]:
# 최적화된 하이퍼파라미터로 모델 재학습
best_params = {
    'n_estimators': round(optimizer.max['params']['n_estimators']),
    'max_depth': round(optimizer.max['params']['max_depth']),
    'max_leaves': round(optimizer.max['params']['max_leaves']),
    'min_child_weight': round(optimizer.max['params']['min_child_weight']),
    'eta': optimizer.max['params']['eta'],
    'scale_pos_weight': optimizer.max['params']['scale_pos_weight'],
    'gamma': optimizer.max['params']['gamma'],
}

best_xgb = XGBClassifier(**best_params, objective='binary:logistic', random_state=42)
best_xgb.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.11644686838124187,
              eval_metric=None, feature_types=None, gamma=3.19560243188283,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=179,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              n_estimators=946, n_jobs=None, num_parallel_tree=None,
              predictor=None, ...)

In [21]:
# Train data 성능 평가
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

y_train_pred = best_xgb.predict(X_train_scaled)
print("<<Train Set Performance>>")
print("Accuracy: ", accuracy_score(y_train, y_train_pred))
print("F1 Score: ", f1_score(y_train, y_train_pred))
print(confusion_matrix(y_train, y_train_pred))

# Test data 성능 평가
y_test_pred = best_xgb.predict(X_test_scaled)
print("<<Test Set Performance>>")
print("Accuracy: ", accuracy_score(y_test, y_test_pred))
print("F1 Score: ", f1_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))

<<Train Set Performance>>
Accuracy:  0.9846015846912842
F1 Score:  0.9565217391304348
[[5453   26]
 [  77 1133]]
<<Test Set Performance>>
Accuracy:  0.8439928272564255
F1 Score:  0.4990403071017275
[[1282   88]
 [ 173  130]]


### Random Forest

In [22]:
# rf 모델 학습
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=42, class_weight='balanced')
rf.fit(X_train_scaled, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=10, n_estimators=500,
                       random_state=42)

In [23]:
# 베이지안 최적화
from sklearn.ensemble import RandomForestClassifier

param_bounds = {
    'n_estimators': (100, 1000),
    'max_depth': (3, 15),
    }

def eval_function(n_estimators, max_depth):
    params = {
    'n_estimators': int(n_estimators),
    'max_depth': int(max_depth),
    }

    rf = RandomForestClassifier(**params, random_state=42)
    rf.fit(X_train_scaled, y_train)
    y_pred = rf.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    return acc

In [24]:
from bayes_opt import BayesianOptimization

optimizer = BayesianOptimization(f=eval_function, pbounds=param_bounds, random_state=42)
optimizer.maximize(init_points=2, n_iter=10)

|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
| 1         | 0.8386    | 7.494     | 955.6     |
| 2         | 0.841     | 11.78     | 638.8     |
| 3         | 0.8428    | 13.07     | 639.1     |
| 4         | 0.8416    | 14.12     | 639.7     |
| 5         | 0.8416    | 14.06     | 637.3     |
| 6         | 0.8422    | 12.99     | 640.0     |
| 7         | 0.8416    | 12.62     | 643.2     |
| 8         | 0.8428    | 14.56     | 644.7     |
| 9         | 0.8422    | 14.94     | 643.9     |
| 10        | 0.8428    | 14.63     | 646.3     |
| 11        | 0.8416    | 12.99     | 647.0     |
| 12        | 0.8422    | 13.32     | 645.1     |


In [25]:
# 최적화된 하이퍼파라미터로 모델 재학습
best_params_rf = {
    'n_estimators': round(optimizer.max['params']['n_estimators']),
    'max_depth': round(optimizer.max['params']['max_depth']),
}

best_rf = RandomForestClassifier(**best_params_rf, random_state=42)
best_rf.fit(X_train_scaled, y_train)

RandomForestClassifier(max_depth=13, n_estimators=639, random_state=42)

In [26]:
# Train data 성능 평가
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

y_train_pred = best_rf.predict(X_train_scaled)
print("<<Train Set Performance>>")
print("Accuracy: ", accuracy_score(y_train, y_train_pred))
print("F1 Score: ", f1_score(y_train, y_train_pred))
print(confusion_matrix(y_train, y_train_pred))

# Test data 성능 평가
y_test_pred = best_rf.predict(X_test_scaled)
print("<<Test Set Performance>>")
print("Accuracy: ", accuracy_score(y_test, y_test_pred))
print("F1 Score: ", f1_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))

<<Train Set Performance>>
Accuracy:  0.9850500822245478
F1 Score:  0.956896551724138
[[5479    0]
 [ 100 1110]]
<<Test Set Performance>>
Accuracy:  0.8427973699940227
F1 Score:  0.3811764705882353
[[1329   41]
 [ 222   81]]


### SVC

In [27]:
# 베이지안 최적화
from sklearn.svm import SVC

param_bounds = {
    'C': (0.1, 10),
    'gamma': (0.1, 10),
    }

def eval_function(C, gamma):

    svc = SVC(kernel='rbf', C=C, gamma=gamma, class_weight='balanced', random_state=42)
    svc.fit(X_train_scaled, y_train)
    y_pred = svc.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    return acc

In [28]:
from bayes_opt import BayesianOptimization

optimizer = BayesianOptimization(f=eval_function, pbounds=param_bounds, random_state=42)
optimizer.maximize(init_points=2, n_iter=5)

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
| 1         | 0.8231    | 3.808     | 9.512     |
| 2         | 0.8452    | 7.347     | 6.027     |
| 3         | 0.8452    | 7.405     | 6.015     |
| 4         | 0.835     | 6.348     | 1.866     |
| 5         | 0.8279    | 10.0      | 8.841     |
| 6         | 0.8494    | 10.0      | 3.585     |
| 7         | 0.7956    | 10.0      | 0.7087    |


In [29]:
# 최적화된 하이퍼파라미터로 모델 재학습
best_params = optimizer.max['params']

best_svc = SVC(**best_params, kernel='rbf', class_weight='balanced', random_state=42)
best_svc.fit(X_train_scaled, y_train)

SVC(C=10.0, class_weight='balanced', gamma=3.5854390547034414, random_state=42)

In [30]:
# Train data 성능 평가
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

y_train_pred = best_svc.predict(X_train_scaled)
print("<<Train Set Performance>>")
print("Accuracy: ", accuracy_score(y_train, y_train_pred))
print("F1 Score: ", f1_score(y_train, y_train_pred))
print(confusion_matrix(y_train, y_train_pred))

# Test data 성능 평가
y_test_pred = best_svc.predict(X_test_scaled)
print("<<Test Set Performance>>")
print("Accuracy: ", accuracy_score(y_test, y_test_pred))
print("F1 Score: ", f1_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))

<<Train Set Performance>>
Accuracy:  1.0
F1 Score:  1.0
[[5479    0]
 [   0 1210]]
<<Test Set Performance>>
Accuracy:  0.8493723849372385
F1 Score:  0.4473684210526316
[[1319   51]
 [ 201  102]]
